In [1]:
import ROOT as rt

from ROOT import TCanvas, TMath, gROOT, TLatex, gStyle
#gROOT.SetStyle('Modern')

Welcome to JupyROOT 6.26/10


In [2]:
%jsroot

In [3]:
gROOT.Reset()
gROOT.SetStyle("Plain")
gStyle.SetPalette(55)
#gStyle.SetCanvasColor('kWhite')
gStyle.SetCanvasBorderMode(0)
gStyle.SetPadBorderMode(0)
gStyle.SetOptFit(1111)
#gStyle.SetOptStat('e')
#gStyle.SetOptStat(111111)
gStyle.SetErrorX(0)
gStyle.SetTitleBorderSize(0)
gStyle.SetTitleX(0.1)
gStyle.SetTitleW(0.8)
gStyle.SetPadLeftMargin(0.18)

In [4]:
def ProcessHisto(hist):
    '''
    h = Histogram
    '''
    col=1
    size=1.4
    style=20

    #gPad.SetTickx();
    #gPad.SetTicky();
    hist.SetMarkerSize(size)
    hist.SetMarkerColor(col)
    hist.SetLineColor(col)
    hist.SetMarkerStyle(style)
    hist.GetYaxis().SetTitleOffset(1.1)
    hist.GetYaxis().SetTitleSize(0.05)
    hist.GetYaxis().SetLabelSize(0.05)
    hist.GetYaxis().SetLabelFont(42)
    hist.GetXaxis().SetLabelFont(42)
    hist.GetYaxis().SetTitleFont(42)
    hist.GetXaxis().SetTitleFont(42)
    hist.GetXaxis().SetTitleOffset(1.0)
    hist.GetXaxis().SetTitleSize(0.045)
    hist.GetXaxis().SetLabelSize(0.045)

In [5]:
# display only number of entries on hist
#gStyle.SetOptStat("e")
#gStyle.SetTitleFont(62, "x")
#gStyle.SetTitleFontSize(0.2)

In [6]:
# read in file with histograms
inFile = rt.TFile.Open("analysis.root")

In [7]:
# load histograms
dPhiHadHad = inFile.dPhiHadHad
dPhiLamLam = inFile.dPhiLamLam
dPhiHadLam = inFile.dPhiHadLam
dPhiLamDivHad = inFile.dPhiLamDivHad

dPhiDEtaHadHad = inFile.dPhiDEtaHadHad
dPhiDEtaLamLam = inFile.dPhiDEtaLamLam
dPhiDEtaHadLam = inFile.dPhiDEtaHadLam
dPhiDEtaLamDivHad = inFile.dPhiDEtaLamDivHad

dPhiCutHadHad = inFile.dPhiCutHadHad
dPhiCutLamLam = inFile.dPhiCutLamLam
dPhiCutHadLam = inFile.dPhiCutHadLam
dPhiCutLamDivHad = inFile.dPhiCutLamDivHad

dPhiDEtaCutHadHad = inFile.dPhiDEtaCutHadHad
dPhiDEtaCutLamLam = inFile.dPhiDEtaCutLamLam
dPhiDEtaCutHadLam = inFile.dPhiDEtaCutHadLam
dPhiDEtaCutLamDivHad = inFile.dPhiDEtaCutLamDivHad

histListNoCut = [dPhiDEtaHadHad, dPhiDEtaLamLam, dPhiDEtaHadLam, dPhiDEtaLamDivHad]
histListCut = [dPhiDEtaCutHadHad, dPhiDEtaCutLamLam, dPhiDEtaCutHadLam, dPhiDEtaCutLamDivHad]

phiHistListNoCut = [dPhiHadHad, dPhiLamLam, dPhiHadLam, dPhiLamDivHad]
phiHistListCut = [dPhiCutHadHad, dPhiCutLamLam, dPhiCutHadLam, dPhiCutLamDivHad]

hlList = [dPhiDEtaCutHadLam, dPhiDEtaHadLam]
hhList = [dPhiDEtaCutHadHad, dPhiDEtaHadHad]


In [8]:
pi = TMath.Pi()

phi_lower = pi / 2
phi_upper = 3 * pi / 2 - 0.0001

eta_lower = -50
eta_upper = 50 - 0.0001 

eta_cut_lower = -1.6
eta_cut_upper = 1.6

In [9]:
int1 = dPhiCutHadHad.Integral(dPhiCutHadHad.FindBin(phi_lower), dPhiCutHadHad.FindBin(phi_upper))
int2 = dPhiHadHad.Integral(dPhiCutHadHad.FindBin(phi_lower), dPhiCutHadHad.FindBin(phi_upper))

print(int1/int2)

0.1980294390420833


For the below ratios, the yield ratio is given by:

yield ratio $ = \frac{(X-X)^{\text{cut}}_{\text{away}}}{(X-X)_{\text{away}}}$

where the X-X represents the counts in the away-side peak of a $\Delta \phi \Delta \eta$ correlation and "cut" refers to a single particle pseudorapidity cut of $|\eta| \leq 0.8$. Crude ratio refers to taking the X-X correlation without single particle cuts, chopping off counts from the away side peak outside of $|\Delta \eta| \leq 1.6$, and taking the ratio with the counts in the entire peak. As a nice formula:

crude ratio $ = \frac{(X-X)^{\Delta \eta \leq 1.6}_{\text{away}}}{(X-X)_{\text{away}}}$

The last ratio is actually for Lambda / Hadron correlations; the histogram name was just inherited from the L-L correlations because of a ROOT trick I used. Also, note there's an ambiguity in how we define where the away-side peak. Here I take the away side peak as existing in $\Delta \phi \in [ \frac{\pi}{2}, \frac{3 \pi}{2}]$, $\Delta \eta \in [-50, 50]$ (take arbitrarily large $\Delta \eta$). 

In [10]:
#arr_awayside = []
for hist, histCut in zip(histListNoCut, histListCut): 
    phi_bin1 = hist.GetXaxis().FindBin(phi_lower)
    phi_bin2 = hist.GetXaxis().FindBin(phi_upper)
    
    eta_bin1 = hist.GetYaxis().FindBin(eta_lower)
    eta_bin2 = hist.GetYaxis().FindBin(eta_upper)

    eta_cut_bin1 = hist.GetYaxis().FindBin(eta_cut_lower)
    eta_cut_bin2 = hist.GetYaxis().FindBin(eta_cut_upper)
    
    away_yield_total = hist.Integral(phi_bin1, phi_bin2, eta_bin1, eta_bin2) 
    away_yield_cut = histCut.Integral(phi_bin1, phi_bin2, eta_bin1, eta_bin2)

    away_yield_crude = hist.Integral(phi_bin1, phi_bin2, eta_cut_bin1, eta_cut_bin2)
    
    yield_ratio = away_yield_cut / away_yield_total
    yield_ratio_crude = away_yield_crude / away_yield_total
    
    print('---------------------------------------------------------------------------')
    print('Histrogram Names: ' + hist.GetTitle() + ' and ' + histCut.GetTitle())
    print("yield ratio = ", yield_ratio)
    print("Crude Ratio", yield_ratio_crude)

---------------------------------------------------------------------------
Histrogram Names: H-H Delta Phi Delta Eta and H-H Delta Phi Delta Eta for Cut
yield ratio =  0.1980294390420833
Crude Ratio 0.7589414433027557
---------------------------------------------------------------------------
Histrogram Names: L-L Delta Phi Delta Eta and L-L Delta Phi Delta Eta for Cut
yield ratio =  0.19135802469135801
Crude Ratio 0.6975308641975309
---------------------------------------------------------------------------
Histrogram Names: H-L Delta Phi Delta Eta and H-L Delta Phi Delta Eta for Cut
yield ratio =  0.18791946308724833
Crude Ratio 0.7531692766592095
---------------------------------------------------------------------------
Histrogram Names: L-L Delta Phi Delta Eta and L-L Delta Phi Delta Eta for Cut
yield ratio =  0.4725957719592063
Crude Ratio 0.5595266752953941


The ratios calculated with the $\Delta \phi$ correlations should equal the ratios calculated with the $\Delta \phi \Delta \eta$ distributions. Let us verify that:

Note: the crude ratio does not make sense for azimuthal correlations. We'd be doing the same thing we did for the $\Delta \phi \Delta \eta$ correlations.

In [11]:
#arr_awayside = []
for hist, histCut in zip(phiHistListNoCut, phiHistListCut): 
    phi_bin1 = hist.GetXaxis().FindBin(phi_lower)
    phi_bin2 = hist.GetXaxis().FindBin(phi_upper)
    
    away_yield_total = hist.Integral(phi_bin1, phi_bin2) 
    away_yield_cut = histCut.Integral(phi_bin1, phi_bin2)
    
    yield_ratio = away_yield_cut / away_yield_total
    yield_ratio_crude = away_yield_crude / away_yield_total
    
    print('---------------------------------------------------------------------------')
    print('Histrogram Names: ' + hist.GetTitle() + ' and ' + histCut.GetTitle())
    print("yield ratio = ", yield_ratio)

---------------------------------------------------------------------------
Histrogram Names: H-H Delta Phi Distribution and H-H Delta Phi for Cut
yield ratio =  0.1980294390420833
---------------------------------------------------------------------------
Histrogram Names: L-L Delta Phi Distribution and L-L Delta Phi for Cut
yield ratio =  0.19135802469135801
---------------------------------------------------------------------------
Histrogram Names: H-L Delta Phi Distribution and H-L Delta Phi for Cut
yield ratio =  0.18791946308724833
---------------------------------------------------------------------------
Histrogram Names: L-L Delta Phi Distribution and L-L Delta Phi for Cut
yield ratio =  0.43679990956551723


This checks out. We also want to compare: 

$\frac{(H-L)^{\text{cut}}_{\text{away}}}{(H-H)^{\text{cut}}_{\text{away}}}$ vs. $\frac{(H-L)_{\text{away}}}{(H-H)_{\text{away}}}$

Basically, how much more does the H-L away-side peak change compared to the H-H peak when we apply single particle pseudorapidity cuts. If the ratio changes a lot, we must correct for detector effects (ie. the ALICE detector being a cylinder) in our analyses. 

In [12]:
away_yield_arr = []
for hl, hh in zip(hlList, hhList): 
    phi_hl_bin1 = hl.GetXaxis().FindBin(phi_lower)
    phi_hl_bin2 = hl.GetXaxis().FindBin(phi_upper)
    
    eta_hl_bin1 = hl.GetYaxis().FindBin(eta_lower)
    eta_hl_bin2 = hl.GetYaxis().FindBin(eta_upper)

    phi_hh_bin1 = hh.GetXaxis().FindBin(phi_lower)
    phi_hh_bin2 = hh.GetXaxis().FindBin(phi_upper)

    eta_hh_bin1 = hh.GetYaxis().FindBin(eta_cut_lower)
    eta_hh_bin2 = hh.GetYaxis().FindBin(eta_cut_upper)
    
    away_yield_hl = hl.Integral(phi_hl_bin1, phi_hl_bin2, eta_hl_bin1, eta_hl_bin2) 
    away_yield_hh = hh.Integral(phi_hh_bin1, phi_hh_bin2, eta_hh_bin1, eta_hh_bin2)

    yield_ratio = away_yield_hl / away_yield_hh
    
    away_yield_arr.append(yield_ratio)
    
    print('---------------------------------------------------------------------------')
    print('Histrogram Names: ' + hl.GetTitle() + ' over ' + hh.GetTitle())
    print("yield ratio = ", yield_ratio)

percent_diff = 100 * (away_yield_arr[0] - away_yield_arr[1]) / away_yield_arr[1]
print('---------------------------------------------------------------------------')
print('The percent difference is ' + str(percent_diff))

---------------------------------------------------------------------------
Histrogram Names: H-L Delta Phi Delta Eta for Cut over H-H Delta Phi Delta Eta for Cut
yield ratio =  0.023370119632755263
---------------------------------------------------------------------------
Histrogram Names: H-L Delta Phi Delta Eta over H-H Delta Phi Delta Eta
yield ratio =  0.03244969812827431
---------------------------------------------------------------------------
The percent difference is -27.98047137334618


In [13]:
cPoster = TCanvas()
#cPoster.SetCanvasSize(2400, 2000)a
cPoster.Divide(2, 2)

cPoster.cd(1)
dPhiDEtaHadHad.SetTitle("Hadron" + "-" + "Hadron Correlation (Theoretical)")
dPhiDEtaHadHad.SetXTitle("#Delta#varphi")
dPhiDEtaHadHad.SetYTitle('#Delta#eta')
dPhiDEtaHadHad.SetStats(0)
#dPhiDEtaHadHad.SetTitleSize(10, "xy")
ProcessHisto(dPhiDEtaHadHad)


dPhiDEtaHadHad.Draw("COLZ")


cPoster.cd(2)
dPhiDEtaHadLam.SetTitle("Hadron-#Lambda Correlation (Theoretical)")
dPhiDEtaHadLam.SetXTitle("#Delta#varphi")
dPhiDEtaHadLam.SetYTitle('#Delta#eta')
dPhiDEtaHadLam.SetStats(0)
#dPhiDEtaHadLam.SetTitleSize(10, "xy")
ProcessHisto(dPhiDEtaHadLam)

dPhiDEtaHadLam.Draw("COLZ")


cPoster.cd(3)
dPhiDEtaCutHadHad.SetTitle("Hadron" + "-" + "Hadron Correlation (|#eta| < 0.8)")
dPhiDEtaCutHadHad.SetXTitle("#Delta#varphi")
dPhiDEtaCutHadHad.SetYTitle('#Delta#eta')
dPhiDEtaCutHadHad.SetStats(0)
#dPhiDEtaCutHadHad.SetTitleSize(10, "xy")
ProcessHisto(dPhiDEtaCutHadHad)

dPhiDEtaCutHadHad.Draw("COLZ")


cPoster.cd(4)
dPhiDEtaCutHadLam.SetTitle("Hadron" + "-" + "#Lambda Correlation (|#eta| < 0.8)")
dPhiDEtaCutHadLam.SetXTitle("#Delta#varphi")
dPhiDEtaCutHadLam.SetYTitle('#Delta#eta')
dPhiDEtaCutHadLam.SetStats(0)
#dPhiDEtaCutHadLam.SetTitleSize(10, "xy")
ProcessHisto(dPhiDEtaCutHadLam)

dPhiDEtaCutHadLam.Draw("COLZ")

cPoster.SaveAs("cPoster.pdf")
cPoster.Draw()

Info in <TCanvas::Print>: pdf file cPoster.pdf has been created


In [14]:
c1 = TCanvas()
c1.SetFillColor(10)

dPhiDEtaHadHad.SetTitle("Hadron" + "-" + "Hadron Correlation (No #eta Cut)")
dPhiDEtaHadHad.SetXTitle("#Delta#varphi")
dPhiDEtaHadHad.SetYTitle('#Delta#eta')

dPhiDEtaHadHad.Draw("COLZ")
c1.Draw()

In [15]:
c2 = TCanvas()

dPhiDEtaLamLam.Draw("COLZ")
c2.Draw()

In [16]:
c3 = TCanvas()

dPhiDEtaHadLam.Draw("COLZ")
dPhiDEtaHadLam.SetTitle("Hadron-#Lambda Correlation (No #eta Cut)")
dPhiDEtaHadLam.SetXTitle("#Delta#varphi")
dPhiDEtaHadLam.SetYTitle('#Delta#eta')
c3.Draw()

In [17]:
c4 = TCanvas()

dPhiDEtaLamDivHad.Draw("COLZ")
c4.Draw()

In [18]:
c5 = TCanvas()

dPhiDEtaCutHadHad.SetTitle("Hadron" + "-" + "Hadron Correlation (|#eta| < 0.8)")
dPhiDEtaCutHadHad.SetXTitle("#Delta#varphi")
dPhiDEtaCutHadHad.SetYTitle('#Delta#eta')


dPhiDEtaCutHadHad.Draw("COLZ")
c5.Draw()

In [19]:
c6 = TCanvas()

dPhiDEtaCutLamLam.Draw("COLZ")
c6.Draw()

In [20]:
c7 = TCanvas()

dPhiDEtaCutHadLam.Draw("COLZ")
dPhiDEtaCutHadLam.SetTitle("Hadron" + "-" + "#Lambda Correlation (|#eta| < 0.8)")
dPhiDEtaCutHadLam.SetXTitle("#Delta#varphi")
dPhiDEtaCutHadLam.SetYTitle('#Delta#eta')
c7.Draw()

In [21]:
c8 = TCanvas()

dPhiDEtaCutLamDivHad.Draw("COLZ")
c8.Draw()

In [22]:
c9 = TCanvas()

c9.Draw()
dPhiHadHad.Draw()

In [23]:
c10 = TCanvas()

c10.Draw()
dPhiCutHadHad.Draw()

In [24]:
c11 = TCanvas()

c11.Draw()
dPhiLamLam.Draw()

In [25]:
c12 = TCanvas()

c12.Draw()
dPhiCutLamLam.Draw()

In [26]:
c13 = TCanvas()

c13.Draw()
dPhiHadLam.Draw()

In [27]:
c14 = TCanvas()

c14.Draw()
dPhiHadLam.Draw()

In [28]:
c12 = TCanvas()

c12.Draw()
dPhiLamDivHad.Draw()